In [11]:
import numpy as np
import pandas as pd
import pandas_ml as pdml

import sklearn
import pickle
import gzip

In [12]:
def load_zipped_pickle(filename):
    # load a zipped compressed pickle file
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object
    
def save_zipped_pickle(obj, filename, protocol=-1):
    # this will default to the highest protocol
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, protocol)

In [49]:
TARGET = ['10_year_cites', '10_year_cites_top1']

In [50]:
TEXT_COLS =['missing_patent_abstract', 'missing_patent_title','flag_has_description_text', 'flag_has_independent_claim_text',
            'patent_abstract', 'patent_title']

In [3]:
nums = pd.read_pickle('/pool001/lraymond/processed_data/pre_analysis_dfs/standardized_linear_classifier_df.p')

In [13]:
index_file = '/pool001/lraymond/patent_data/index_files/sample_pat_nums_all_numerical.gzip'

valid_patents2 = load_zipped_pickle(index_file)

In [30]:
m = pd.isnull(valid_patents2['patent_abstract'])
valid_patents2.loc[m, 'missing_patent_abstract'] = 1

m = pd.isnull(valid_patents2['patent_title'])
valid_patents2.loc[m, 'missing_patent_title'] = 1


from sklearn import svm
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler

In [61]:
txt_pats = valid_patents2.loc[(
    (valid_patents2.missing_patent_title==0) & (
        valid_patents2.missing_patent_abstract==0) & (valid_patents2.flag_has_independent_claim_text==1) & (
            valid_patents2.flag_has_description_text==1)), 
            ['patent_number', 'patent_abstract', 'patent_title']+TARGET]

In [74]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn import model_selection

rus = RandomUnderSampler(sampling_strategy=.5, return_indices=True, random_state=0)
Y_col = ['10_year_cites_top1']
X_cols = ['patent_abstract']

X = txt_pats[['patent_abstract']].as_matrix()
Y = txt_pats[Y_col[0]].as_matrix()


X_resampled, y_resampled, ind = rus.fit_resample(X, Y)

X_train, X_test, y_train, y_test, pat_nums_train, pat_nums_test = model_selection.train_test_split(
    X_resampled, y_resampled, ind, random_state=0, shuffle=True)


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.svm import SVC

text_clf = Pipeline([
    ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
    ('svc-sig', SVC(kernel='sigmoid', probability=True)),
])


text_clf.fit(X_train.reshape(-1,), y_train)

predicted = text_clf.predict(X_test.reshape(-1,))

In [93]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted, target_names=['no', 'yes']))

             precision    recall  f1-score   support

         no       0.68      1.00      0.81       789
        yes       0.00      0.00      0.00       375

avg / total       0.46      0.68      0.55      1164



/home/software/sloan/local/lib/py36/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
